In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from time import sleep
from urllib.parse import quote
import threading
from queue import Queue
import pandas as pd

In [2]:
def get_href(elements):
    hrefs = []
    for elemento in elements:
        href = elemento.get_attribute('href')
        hrefs.append(href)

    return hrefs

def consultar(navegador, beneficio, start_date, end_date, municipio = 'Tanguá'):
    municipio = quote(municipio)
    navegador.get(f'https://portaldatransparencia.gov.br/beneficios/{beneficio}?de={start_date}&ate={end_date}&nomeMunicipio={municipio}&ordenarPor=nis&direcao=asc')

def consultar_Bolsa_Familia(navegador, beneficio, start_date, end_date, municipio = 'Tanguá'):
    # start_date = quote(start_date, safe='')
    # end_date = quote(end_date, safe='')
    municipio = quote(municipio)
    return f'https://portaldatransparencia.gov.br/beneficios/{beneficio}?de={start_date}&ate={end_date}&tipoBeneficio=1&nomeMunicipio={municipio}&ordenarPor=nis&direcao=asc'


In [6]:
driver = webdriver.Chrome()
consultar(driver,'bolsa-familia', '01/01/2016', '31/12/2016', 'Tanguá')
# inicia a pagian e espera 10 segundos para os dados carregarem
sleep(5)

# Clica no botão de ver paginação completa
# driver.find_element(By.XPATH, '//*[@id="lista_wrapper"]/div/div[3]/div[3]/button').click()
driver.find_element(By.CLASS_NAME, 'botao__gera_paginacao_completa').click()



# Clica no select para aumentar o numero de dados na pagina
elemento_dropdown  = driver.find_element(By.XPATH, '//*[@id="lista_length"]/label/select')
Select(elemento_dropdown).select_by_value('50')

sleep(5)
# Verifica quantas paginas possuem
quantPaginas = driver.find_element(By.XPATH, '//*[@id="lista_info"]').text.split()[-1]

# Crie uma lista para armazenar os valores de atributo href
hrefs = []

for i in range(int(quantPaginas)):
    # Busca os lnks detalhar
    elements = driver.find_elements(By.CLASS_NAME, 'linkRendered')

    # Itere sobre cada elemento e obtenha o valor do atributo href
    hrefs.extend(get_href(elements))

    # Clica no botão proximo  
    driver.find_element(By.ID, 'lista_next').click()

    
    sleep(7)

driver.quit()

In [43]:
# df = pd.DataFrame(hrefs, columns=['Detalhes_link'])

'https://portaldatransparencia.gov.br/beneficios/bolsa-familia?de=01/01/2016&ate=31/12/2016&tipoBeneficio=1&nomeMunicipio=Tangu%C3%A1&ordenarPor=nis&direcao=asc'

In [46]:
# df.to_csv('2016.csv')

# Teste Threads

In [10]:
def getQuantPaginas():
    driver = webdriver.Chrome()
    consultar(driver,'bolsa-familia', '01/01/2016', '31/12/2016', 'Tanguá')
    # inicia a pagian e espera 10 segundos para os dados carregarem
    sleep(5)

    # Clica no botão de ver paginação completa
    # driver.find_element(By.XPATH, '//*[@id="lista_wrapper"]/div/div[3]/div[3]/button').click()
    driver.find_element(By.CLASS_NAME, 'botao__gera_paginacao_completa').click()



    # Clica no select para aumentar o numero de dados na pagina
    elemento_dropdown  = driver.find_element(By.XPATH, '//*[@id="lista_length"]/label/select')
    Select(elemento_dropdown).select_by_value('50')

    sleep(5)
    # Verifica quantas paginas possuem
    quantPaginas = driver.find_element(By.XPATH, '//*[@id="lista_info"]').text.split()[-1]
    driver.quit()
    return int(quantPaginas)

In [11]:
# Função que será executada em cada thread
def coletar_dados(start_page, end_page, beneficio, start_date, end_date, municipio='Tanguá'):
    driver = webdriver.Chrome()
    driver.execute_script("window.open('about:blank', 'new_tab');")
    driver.switch_to.window(driver.window_handles[-1])
    consultar(driver, beneficio, start_date, end_date, municipio)
    
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'botao__gera_paginacao_completa'))
        )
    finally:
        driver.find_element(By.CLASS_NAME, 'botao__gera_paginacao_completa').click()

    sleep(5)
    elemento_dropdown = driver.find_element(By.XPATH, '//*[@id="lista_length"]/label/select')
    Select(elemento_dropdown).select_by_value('50')

    sleep(4)
    driver.find_element(By.ID, 'paginas-selecao-1-lista').send_keys(start_page)
    driver.find_element(By.ID, 'botao-ir-para-a-pagina-lista').click()
    sleep(7)
    hrefs = []
    for _ in range(start_page, end_page + 1):
        elements = driver.find_elements(By.CLASS_NAME, 'linkRendered')
        hrefs.extend(get_href(elements))
        driver.find_element(By.ID, 'lista_next').click()
        sleep(8)

    driver.quit()

    if results_queue:
        results_queue.put(hrefs)


# Execução das threads
beneficio = 'bolsa-familia'
start_date = '01/01/2016'
end_date = '31/12/2016'
municipio = 'Tanguá'

num_threads = 6
quantPaginas = getQuantPaginas()
pages_per_thread = quantPaginas // num_threads

threads = []
results_queue = Queue()

for i in range(num_threads):
    start_page = i * pages_per_thread + 1
    end_page = start_page + pages_per_thread - 1 if i < num_threads - 1 else quantPaginas
    thread = threading.Thread(target=coletar_dados, args=(start_page, end_page, beneficio, start_date, end_date, municipio))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

# Recupera os resultados das threads
hrefs = []
while not results_queue.empty():
    hrefs.extend(results_queue.get())


# Teste de extrair quantidade da tabela

In [15]:
driver = webdriver.Chrome()

driver.get('https://portaldatransparencia.gov.br/beneficios/bolsa-familia/293703902?ordenarPor=mesReferencia&direcao=desc')
sleep(3)
# Localize a tabela
tabela = driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos')

# Obtenha todas as linhas da tabela
linhas = tabela.find_elements(By.TAG_NAME, 'tr')


dados = []

# Itere sobre as linhas
for linha in linhas:
    # Obtenha todas as células da linha
    celulas = linha.find_elements(By.TAG_NAME, 'td')
    
    # Extraia os dados das células
    dados_linha = [celula.text for celula in celulas]
    
    # Armazene os dados
    dados.append(dados_linha)

# Feche o navegador
# driver.quit()

# Faça algo com os dados coletados
print(dados)


[[], ['10/2021', '10/2021', 'RJ', 'TANGUÁ', '2', '147,00'], ['09/2021', '09/2021', 'RJ', 'TANGUÁ', '2', '147,00'], ['08/2021', '08/2021', 'RJ', 'TANGUÁ', '2', '147,00'], ['07/2021', '07/2021', 'RJ', 'TANGUÁ', '1', '147,00'], ['06/2021', '06/2021', 'RJ', 'TANGUÁ', '1', '147,00'], ['05/2021', '05/2021', 'RJ', 'TANGUÁ', '2', '178,00'], ['04/2021', '04/2021', 'RJ', 'TANGUÁ', '2', '178,00'], ['03/2021', '03/2021', 'RJ', 'TANGUÁ', '2', '178,00'], ['02/2021', '02/2021', 'RJ', 'TANGUÁ', '2', '178,00'], ['01/2021', '01/2021', 'RJ', 'TANGUÁ', '2', '171,00'], ['12/2020', '12/2020', 'RJ', 'TANGUÁ', '3', '219,00'], ['11/2020', '11/2020', 'RJ', 'TANGUÁ', '3', '219,00'], ['10/2020', '10/2020', 'RJ', 'TANGUÁ', '3', '219,00'], ['09/2020', '09/2020', 'RJ', 'TANGUÁ', '3', '219,00'], ['08/2020', '08/2020', 'RJ', 'TANGUÁ', '2', '178,00']]


In [16]:
dados

[[],
 ['10/2021', '10/2021', 'RJ', 'TANGUÁ', '2', '147,00'],
 ['09/2021', '09/2021', 'RJ', 'TANGUÁ', '2', '147,00'],
 ['08/2021', '08/2021', 'RJ', 'TANGUÁ', '2', '147,00'],
 ['07/2021', '07/2021', 'RJ', 'TANGUÁ', '1', '147,00'],
 ['06/2021', '06/2021', 'RJ', 'TANGUÁ', '1', '147,00'],
 ['05/2021', '05/2021', 'RJ', 'TANGUÁ', '2', '178,00'],
 ['04/2021', '04/2021', 'RJ', 'TANGUÁ', '2', '178,00'],
 ['03/2021', '03/2021', 'RJ', 'TANGUÁ', '2', '178,00'],
 ['02/2021', '02/2021', 'RJ', 'TANGUÁ', '2', '178,00'],
 ['01/2021', '01/2021', 'RJ', 'TANGUÁ', '2', '171,00'],
 ['12/2020', '12/2020', 'RJ', 'TANGUÁ', '3', '219,00'],
 ['11/2020', '11/2020', 'RJ', 'TANGUÁ', '3', '219,00'],
 ['10/2020', '10/2020', 'RJ', 'TANGUÁ', '3', '219,00'],
 ['09/2020', '09/2020', 'RJ', 'TANGUÁ', '3', '219,00'],
 ['08/2020', '08/2020', 'RJ', 'TANGUÁ', '2', '178,00']]

In [14]:
len(linhas)

16

# Tratamento

In [31]:
df = pd.read_csv('2016-1.csv')
df.reset_index(drop=True, inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)
df.to_csv('bolsa-familia-2016.csv')